<a href="https://colab.research.google.com/github/melvin97n/TMDB-prediction/blob/master/TMDB_prediction_draft.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import seaborn as sns
from tqdm import tqdm
from scipy import stats
from scipy.stats import norm, skew #for some statistics
pd.set_option('display.float_format', lambda x: '{:.3f}'.format(x))
%matplotlib inline
import os
import re

In [0]:
!pip install kaggle
from google.colab import files
files.upload()

Saving kaggle.json to kaggle (1).json


{'kaggle.json': b'{"username":"melvin97n","key":"e8805b38b4617ec29d832f8bbdfeaab8"}'}

In [0]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!ls ~/.kaggle
!ls -l ~/.kaggle
!cat ~/.kaggle/kaggle.json
!pip install -q kaggle
!pip install -q kaggle-cli

kaggle.json
total 4
-rw------- 1 root root 65 Aug 12 11:43 kaggle.json
{"username":"melvin97n","key":"e8805b38b4617ec29d832f8bbdfeaab8"}

In [0]:
!kaggle competitions download -c tmdb-box-office-prediction

sample_submission.csv: Skipping, found more recently modified local copy (use --force to force download)
train.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
test.csv.zip: Skipping, found more recently modified local copy (use --force to force download)


In [0]:
pwd

'/content'

In [0]:
test=pd.read_csv("test.csv.zip")
train=pd.read_csv("train.csv.zip")
train_x=train.drop('revenue',1)
df=pd.concat([train_x,test])
df.head(5)

In [0]:
df=pd.read_csv("train.csv.zip")

In [0]:
df.isnull().sum(axis=0)

id                          0
belongs_to_collection    2396
budget                      0
genres                      7
homepage                 2054
imdb_id                     0
original_language           0
original_title              0
overview                    8
popularity                  0
poster_path                 1
production_companies      156
production_countries       55
release_date                0
runtime                     2
spoken_languages           20
status                      0
tagline                   597
title                       0
Keywords                  276
cast                       13
crew                       16
revenue                     0
dtype: int64

In [0]:
df['belongs_to_collection'][df['belongs_to_collection'].notnull()]='Yes'

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [0]:
df['belongs_to_collection']=df['belongs_to_collection'].fillna("No")

In [0]:
x=[]
for index,row in df.iterrows():
  try:
    d=eval(df['genres'][index])
    b=[]
    for i in range(len(d)):
      b.append(list(d[i].values())[1])
  except:
    pass
  x.append(b)
df['genres']=x



In [0]:
df.head()

,id,belongs_to_collection,budget,genres,homepage,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,runtime,spoken_languages,status,tagline,title,Keywords,cast,crew,revenue
0,1,Yes,14000000,[Comedy],NaN,tt2637294,en,Hot Tub Time Machine 2,"When Lou, who has become the ""father of the In...",6.575,/tQtWuwvMf0hCc2QR2tkolwl7c3c.jpg,"[{'name': 'Paramount Pictures', 'id': 4}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",2/20/15,93.000,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,The Laws of Space and Time are About to be Vio...,Hot Tub Time Machine 2,"[{'id': 4379, 'name': 'time travel'}, {'id': 9...","[{'cast_id': 4, 'character': 'Lou', 'credit_id...","[{'credit_id': '59ac067c92514107af02c8c8', 'de...",12314651
1,2,Yes,40000000,"[Comedy, Drama, Family, Romance]",NaN,tt0368933,en,The Princess Diaries 2: Royal Engagement,Mia Thermopolis is now a college graduate and ...,8.249,/w9Z7A0GHEhIp7etpj0vyKOeU1Wx.jpg,"[{'name': 'Walt Disney Pictures', 'id': 2}]","[{'iso_3166_1': 'US', 'name': 'United States o...",8/6/04,113.000,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,It can take a lifetime to find true love; she'...,The Princess Diaries 2: Royal Engagement,"[{'id': 2505, 'name': 'coronation'}, {'id': 42...","[{'cast_id': 1, 'character': 'Mia Thermopolis'...","[{'credit_id': '52fe43fe9251416c7502563d', 'de...",95149435
2,3,No,3300000,[Drama],http://sonyclassics.com/whiplash/,tt2582802,en,Whiplash,"Under the direction of a ruthless instructor, ...",64.300,/lIv1QinFqz4dlp5U4lQ6HaiskOZ.jpg,"[{'name': 'Bold Films', 'id': 2266}, {'name': ...","[{'iso_3166_1': 'US', 'name': 'United States o...",10/10/14,105.000,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,The road to greatness can take you to the edge.,Whiplash,"[{'id': 1416, 'name': 'jazz'}, {'id': 1523, 'n...","[{'cast_id': 5, 'character': 'Andrew Neimann',...","[{'credit_id': '54d5356ec3a3683ba0000039', 'de...",13092000
3,4,No,1200000,"[Thriller, Drama]",http://kahaanithefilm.com/,tt1821480,hi,Kahaani,Vidya Bagchi (Vidya Balan) arrives in Kolkata ...,3.175,/aTXRaPrWSinhcmCrcfJK17urp3F.jpg,NaN,"[{'iso_3166_1': 'IN', 'name': 'India'}]",3/9/12,122.000,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,NaN,Kahaani,"[{'id': 10092, 'name': 'mystery'}, {'id': 1054...","[{'cast_id': 1, 'character': 'Vidya Bagchi', '...","[{'credit_id': '52fe48779251416c9108d6eb', 'de...",16000000
4,5,No,0,"[Action, Thriller]",NaN,tt1380152,ko,마린보이,Marine Boy is the story of a former national s...,1.148,/m22s7zvkVFDU9ir56PiiqIEWFdT.jpg,NaN,"[{'iso_3166_1': 'KR', 'name': 'South Korea'}]",2/5/09,118.000,"[{'iso_639_1': 'ko', 'name': '한국어/조선말'}]",Released,NaN,Marine Boy,NaN,"[{'cast_id': 3, 'character': 'Chun-soo', 'cred...","[{'credit_id': '52fe464b9251416c75073b43', 'de...",3923970


In [0]:
df['cast'][0]

"[{'cast_id': 4, 'character': 'Lou', 'credit_id': '52fe4ee7c3a36847f82afae7', 'gender': 2, 'id': 52997, 'name': 'Rob Corddry', 'order': 0, 'profile_path': '/k2zJL0V1nEZuFT08xUdOd3ucfXz.jpg'}, {'cast_id': 5, 'character': 'Nick', 'credit_id': '52fe4ee7c3a36847f82afaeb', 'gender': 2, 'id': 64342, 'name': 'Craig Robinson', 'order': 1, 'profile_path': '/tVaRMkJXOEVhYxtnnFuhqW0Rjzz.jpg'}, {'cast_id': 6, 'character': 'Jacob', 'credit_id': '52fe4ee7c3a36847f82afaef', 'gender': 2, 'id': 54729, 'name': 'Clark Duke', 'order': 2, 'profile_path': '/oNzK0umwm5Wn0wyEbOy6TVJCSBn.jpg'}, {'cast_id': 7, 'character': 'Adam Jr.', 'credit_id': '52fe4ee7c3a36847f82afaf3', 'gender': 2, 'id': 36801, 'name': 'Adam Scott', 'order': 3, 'profile_path': '/5gb65xz8bzd42yjMAl4zwo4cvKw.jpg'}, {'cast_id': 8, 'character': 'Hot Tub Repairman', 'credit_id': '52fe4ee7c3a36847f82afaf7', 'gender': 2, 'id': 54812, 'name': 'Chevy Chase', 'order': 4, 'profile_path': '/svjpyYtPwtjvRxX9IZnOmOkhDOt.jpg'}, {'cast_id': 9, 'character

In [0]:
df['genres'][470]=['Adventure']
df['genres'][1622]=[ 'Drama', 'Comedy']
df['genres'][1814]=['Comedy']
df['genres'][1819]=['Romance', 'Drama']
df['genres'][2423]=['Action', 'Drama', 'Romance']
df['genres'][2686]=['Thriller']
df['genres'][2900]=['Drama', 'Fantasy', 'Mystery']


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-p

In [0]:
df.drop(['homepage','imdb_id','overview','poster_path','spoken_languages','Keywords','tagline','title','original_title'],axis=1,inplace=True)

In [0]:
df['runtime'][1335]=130
df['runtime'][2302]=87

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [0]:
df.isnull().sum(axis=0)

id                         0
belongs_to_collection      0
budget                     0
genres                     0
original_language          0
popularity                 0
production_companies     156
production_countries      55
release_date               0
runtime                    0
status                     0
cast                      13
crew                      16
revenue                    0
dtype: int64

In [0]:
x=[]
for index,row in df.iterrows():
  try:
    d=eval(df['production_companies'][index])
    b=[]
    for i in range(len(d)):
      b.append(list(d[i].values())[0])
  except:
    b=[]
  x.append(b)
df['production_companies']=x


In [0]:
studios=df['production_companies'].apply(pd.Series).stack().unique()
studios=studios[:30]
studios

array(['Paramount Pictures', 'United Artists',
       'Metro-Goldwyn-Mayer (MGM)', 'Walt Disney Pictures', 'Bold Films',
       'Blumhouse Productions', 'Right of Way Films',
       'Ghost House Pictures', 'North Box Productions',
       'Jim Henson Productions', 'Jim Henson Company, The',
       'Castle Rock Entertainment',
       'Twentieth Century Fox Film Corporation', 'Amercent Films',
       'American Entertainment Partners L.P.',
       'Interscope Communications', 'DreamWorks SKG',
       'Jinks/Cohen Company', 'Double Feature Films', 'Jersey Films',
       'Nina Saxon Film Design', 'Cruise/Wagner Productions',
       'Amblin Entertainment', 'Blue Tulip Productions',
       'Ronald Shusett/Gary Goldman', 'Digital Image Associates',
       'Hypnopolis', 'Craven-Maddalena Films', 'BenderSpink', 'BBC Films'],
      dtype=object)

In [0]:
x=[]
for index,row in df.iterrows():
  try:
    d=eval(df['production_countries'][index])
    b=[]
    for i in range(len(d)):
      b.append(list(d[i].values())[0])
  except:
    b=[]
  x.append(b)
df['production_countries']=x




In [0]:
x=[]
for index,row in df.iterrows():
  try:
    d=eval(df['cast'][index])
    b=[]
    for i in range(len(d)):
      b.append(list(d[i].values())[0])
  except:
    b=[]
  x.append(b)
df['cast']=x


In [0]:
casts=df['cast'].apply(pd.Series).stack().unique()
casts=casts[:30]

In [0]:
total_crew=[]
for index,row in df.iterrows():
  hi=df['crew'][index]
  crew=[]
  try:
    ho=hi.rsplit("'Director', 'name': '")[1]
    he=ho.split("', ")
    crew.append(he[0])
  except:
    crew.append("N/A")

  try:
    ho=hi.rsplit("Producer\', \'name\': \'")[1]
    he=ho.split("', ")
    crew.append(he[0])
  except:
    crew.append("N/A")
  try:
    ho=hi.rsplit("'Director of Photography', 'name': '")[1]
    he=ho.split("', ")
    crew.append(he[0])
  except:
    crew.append("N/A")
  total_crew.append(crew)
total_crew
df['crew']=total_crew
df["director"] = df["crew"].str[0]
df["producer"] = df["crew"].str[1]
df["Director_of_photography"] = df["crew"].str[2]

In [0]:
df.head()

,id,belongs_to_collection,budget,genres,original_language,popularity,production_companies,production_countries,release_date,runtime,status,cast,crew,revenue,director,producer,Director_of_photography
0,1,Yes,14000000,[Comedy],en,6.575,"[Paramount Pictures, United Artists, Metro-Gol...",[US],2/20/15,93.000,Released,"[4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17,...","[Steve Pink, Andrew Panay, Declan Quinn]",12314651,Steve Pink,Andrew Panay,Declan Quinn
1,2,Yes,40000000,"[Comedy, Drama, Family, Romance]",en,8.249,[Walt Disney Pictures],[US],8/6/04,113.000,Released,"[1, 2, 3, 4, 5, 6, 7, 17, 18, 19, 21, 20, 22, ...","[Garry Marshall, Whitney Houston, Charles Minsky]",95149435,Garry Marshall,Whitney Houston,Charles Minsky
2,3,No,3300000,[Drama],en,64.300,"[Bold Films, Blumhouse Productions, Right of W...",[US],10/10/14,105.000,Released,"[5, 6, 11, 12, 13, 14, 15, 22, 23, 24, 25, 26,...","[Damien Chazelle, Jason Reitman, Sharone Meir]",13092000,Damien Chazelle,Jason Reitman,Sharone Meir
3,4,No,1200000,"[Thriller, Drama]",hi,3.175,[],[IN],3/9/12,122.000,Released,"[1, 5, 6, 7, 8, 9, 10]","[Sujoy Ghosh, Sujoy Ghosh, N/A]",16000000,Sujoy Ghosh,Sujoy Ghosh,N/A
4,5,No,0,"[Action, Thriller]",ko,1.148,[],[KR],2/5/09,118.000,Released,"[3, 4, 5, 7]","[Jong-seok Yoon, N/A, N/A]",3923970,Jong-seok Yoon,N/A,N/A


In [0]:
df=df.drop('crew',1)

In [0]:
cols=df.columns
cols

In [0]:
producers=df['producer'].apply(pd.Series).stack().unique()
producers=producers[:30]
dop=df['Director_of_photography'].apply(pd.Series).stack().unique()
dop=dop[:30]
directors=df['director'].apply(pd.Series).stack().unique()
directors=directors[:30]

In [0]:
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()
df = df.join(pd.DataFrame(mlb.fit_transform(df.pop('genres')),
                          columns=mlb.classes_,
                          index=df.index))

In [0]:
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()
df = df.join(pd.DataFrame(mlb.fit_transform(df.pop('production_companies')),
                          columns=mlb.classes_,
                          index=df.index))

In [0]:

from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()
df = df.join(pd.DataFrame(mlb.fit_transform(df.pop('cast')),
                          columns=mlb.classes_,
                          index=df.index))

In [0]:
column=list(cols)+list(studios)+list(casts)
column.remove("genres")
column.remove("production_companies")
column.remove("cast")
df=df[column]

In [0]:

from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
df = df.join(pd.DataFrame(mlb.fit_transform(df.pop('production_countries')),
                          columns=mlb.classes_,
                          index=df.index))


In [0]:
from sklearn.preprocessing import LabelEncoder
lbl=LabelEncoder()
df['belongs_to_collection']=lbl.fit_transform(df['belongs_to_collection'])
from sklearn.preprocessing import LabelEncoder
lbl=LabelEncoder()
df['status']=lbl.fit_transform(df['status'])

In [0]:
language=pd.get_dummies(df.original_language)
language.drop('ar' ,1)
df1=pd.concat([df,language],1)
df1.drop('original_language',1, inplace=True)



In [0]:
dire=pd.get_dummies(df.director)
dire=dire[directors]
df1=pd.concat([df1,dire],1)
df1.drop('director',1, inplace=True)

pros=pd.get_dummies(df.producer)
pros=pros[producers]
df1=pd.concat([df1,pros],1)
df1.drop('producer',1, inplace=True)

phos=pd.get_dummies(df.Director_of_photography)
phos=phos[dop]
df1=pd.concat([df1,phos],1)
df1.drop('Director_of_photography',1, inplace=True)


In [0]:
import datetime as dt
for i in df1.index:
       if pd.notna(df1['release_date'][i]): 
            df1['release_date'][i]=dt.datetime.strptime(df1['release_date'][i],'%m/%d/%y')

In [0]:
df1.drop("N/A",1)
df1['release_date'][i]=dt.datetime.strptime(df1['release_date'][i],'%m/%d/%y')

In [0]:
year = lambda x: dt.datetime.strptime(x, "%m/%d/%y" ).year
df1['year'] = df1['release_date'].map(year)
month = lambda x: dt.datetime.strptime(x, "%m/%d/%y" ).month
df1['month'] = df1['release_date'].map(month)
day = lambda x: dt.datetime.strptime(x, "%m/%d/%y" ).day
df1['day'] = df1['release_date'].map(day)
day_of_week = lambda x: dt.datetime.strptime(x, "%m/%d/%y" ).weekday()
df1['week_day']=df1['release_date'].map(day_of_week)

In [0]:
df1['year']=df1['year'].astype(str)
df1['month']='month' + df1['month'].astype(str)
df1['day']='day' + df1['day'].astype(str)
df1['week_day']='week_day' + df1['week_day'].astype(str)


In [0]:
months=pd.get_dummies(df1.month)
months.drop('month2' ,1)
df1=pd.concat([df1,months],1)
df1.drop('month',1, inplace=True)

years=pd.get_dummies(df1.year)
years.drop('2015' ,1)
df1=pd.concat([df1,years],1)
df1.drop('year',1, inplace=True)

days=pd.get_dummies(df1.day)
days.drop('day1' ,1)
df1=pd.concat([df1,days],1)
df1.drop('day',1, inplace=True)

weekdays=pd.get_dummies(df1.week_day)
weekdays.drop('week_day1' ,1)
df1=pd.concat([df1,weekdays],1)
df1.drop('week_day',1, inplace=True)



In [0]:
df1.drop("release_date",1,inplace=True)
df1.head()

In [0]:
df1 = df1[df1.budget!=0]
df1.shape

(2188, 407)

In [0]:
y=df1['revenue']
df1.drop('revenue',1,inplace=True)
df1['revenue']=y
df1.head()

,id,belongs_to_collection,budget,popularity,runtime,status,Paramount Pictures,United Artists,Metro-Goldwyn-Mayer (MGM),Walt Disney Pictures,Bold Films,Blumhouse Productions,Right of Way Films,Ghost House Pictures,North Box Productions,Jim Henson Productions,"Jim Henson Company, The",Castle Rock Entertainment,Twentieth Century Fox Film Corporation,Amercent Films,American Entertainment Partners L.P.,Interscope Communications,DreamWorks SKG,Jinks/Cohen Company,Double Feature Films,Jersey Films,Nina Saxon Film Design,Cruise/Wagner Productions,Amblin Entertainment,Blue Tulip Productions,Ronald Shusett/Gary Goldman,Digital Image Associates,Hypnopolis,Craven-Maddalena Films,BenderSpink,BBC Films,4,5,6,7,...,2068,day1,day10,day11,day12,day13,day14,day15,day16,day17,day18,day19,day2,day20,day21,day22,day23,day24,day25,day26,day27,day28,day29,day3,day30,day31,day4,day5,day6,day7,day8,day9,week_day0,week_day1,week_day2,week_day3,week_day4,week_day5,week_day6,revenue
0,1,1,14000000,6.575,93.000,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,12314651
1,2,1,40000000,8.249,113.000,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,95149435
2,3,0,3300000,64.300,105.000,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,...,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,13092000
3,4,0,1200000,3.175,122.000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,16000000
5,6,0,8000000,0.743,83.000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,3261638


In [0]:
X=df1.drop('revenue',1)

In [0]:
y=np.log1p(y)

In [0]:
X['budget']=np.log1p(X['budget'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [0]:
X['popularity']=np.log1p(X['popularity'])
X['runtime']=np.log1p(X['runtime'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [0]:
X.head()

,id,belongs_to_collection,budget,popularity,runtime,status,Paramount Pictures,United Artists,Metro-Goldwyn-Mayer (MGM),Walt Disney Pictures,Bold Films,Blumhouse Productions,Right of Way Films,Ghost House Pictures,North Box Productions,Jim Henson Productions,"Jim Henson Company, The",Castle Rock Entertainment,Twentieth Century Fox Film Corporation,Amercent Films,American Entertainment Partners L.P.,Interscope Communications,DreamWorks SKG,Jinks/Cohen Company,Double Feature Films,Jersey Films,Nina Saxon Film Design,Cruise/Wagner Productions,Amblin Entertainment,Blue Tulip Productions,Ronald Shusett/Gary Goldman,Digital Image Associates,Hypnopolis,Craven-Maddalena Films,BenderSpink,BBC Films,4,5,6,7,...,2067,2068,day1,day10,day11,day12,day13,day14,day15,day16,day17,day18,day19,day2,day20,day21,day22,day23,day24,day25,day26,day27,day28,day29,day3,day30,day31,day4,day5,day6,day7,day8,day9,week_day0,week_day1,week_day2,week_day3,week_day4,week_day5,week_day6
0,1,1,16.455,2.025,4.543,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,2,1,17.504,2.225,4.736,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0
2,3,0,15.009,4.179,4.663,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,4,0,13.998,1.429,4.812,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0
5,6,0,15.895,0.556,4.431,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0


In [0]:
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error 
from matplotlib import pyplot as plt
import seaborn as sb
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import warnings 
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)
from xgboost import XGBRegressor

Using TensorFlow backend.


In [0]:
NN_model = Sequential()

# The Input Layer :
NN_model.add(Dense(406, kernel_initializer='normal',input_dim = X.shape[1], activation='relu'))

# The Hidden Layers :
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))
NN_model.add(Dense(256, kernel_initializer='normal',activation='relu'))

# The Output Layer :
NN_model.add(Dense(1, kernel_initializer='normal',activation='linear'))

# Compile the network :
NN_model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])
NN_model.summary()

W0809 07:23:59.889520 139889839216512 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0809 07:23:59.933638 139889839216512 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0809 07:23:59.941443 139889839216512 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4115: The name tf.random_normal is deprecated. Please use tf.random.normal instead.

W0809 07:24:00.016614 139889839216512 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 406)               165242    
_________________________________________________________________
dense_2 (Dense)              (None, 256)               104192    
_________________________________________________________________
dense_3 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_4 (Dense)              (None, 256)               65792     
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 257       
Total params: 401,275
Trainable params: 401,275
Non-trainable params: 0
_________________________________________________________________


In [0]:
NN_model.fit(X, y, epochs=100, batch_size=32, validation_split = 0.2)


W0809 07:24:00.237253 139889839216512 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.

W0809 07:24:00.405346 139889839216512 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:973: The name tf.assign is deprecated. Please use tf.compat.v1.assign instead.



Train on 1750 samples, validate on 438 samples
Epoch 1/100
1750/1750 [==============================] - 4s 2ms/step - loss: 7.8511 - mean_absolute_error: 7.8511 - val_loss: 8.3362 - val_mean_absolute_error: 8.3362
Epoch 2/100
1750/1750 [==============================] - 0s 189us/step - loss: 3.6843 - mean_absolute_error: 3.6843 - val_loss: 6.5886 - val_mean_absolute_error: 6.5886
Epoch 3/100
1750/1750 [==============================] - 0s 207us/step - loss: 2.6515 - mean_absolute_error: 2.6515 - val_loss: 1.6105 - val_mean_absolute_error: 1.6105
Epoch 4/100
1750/1750 [==============================] - 0s 193us/step - loss: 1.7692 - mean_absolute_error: 1.7692 - val_loss: 2.2782 - val_mean_absolute_error: 2.2782
Epoch 5/100
1750/1750 [==============================] - 0s 191us/step - loss: 1.7653 - mean_absolute_error: 1.7653 - val_loss: 6.5250 - val_mean_absolute_error: 6.5250
Epoch 6/100
1750/1750 [==============================] - 0s 202us/step - loss: 1.7162 - mean_absolute_error: 1

In [0]:
NN_model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])

In [0]:
ans=NN_model.predict(Xtest)

ValueError: ignored

In [0]:
mean_absolute_error(ans,y)

1.0210272508150038

For TEST SET

In [0]:
df=pd.read_csv("test.csv.zip")
df['belongs_to_collection'][df['belongs_to_collection'].notnull()]='Yes'
df['belongs_to_collection']=df['belongs_to_collection'].fillna("No")
x=[]
for index,row in df.iterrows():
  try:
    d=eval(df['genres'][index])
    b=[]
    for i in range(len(d)):
      b.append(list(d[i].values())[1])
  except:
    pass
  x.append(b)
df['genres']=x
df.drop(['homepage','imdb_id','overview','poster_path','spoken_languages','Keywords','tagline','title','original_title'],axis=1,inplace=True)
x=[]
for index,row in df.iterrows():
  try:
    d=eval(df['production_companies'][index])
    b=[]
    for i in range(len(d)):
      b.append(list(d[i].values())[0])
  except:
    b=[]
  x.append(b)
df['production_companies']=x
x=[]
for index,row in df.iterrows():
  try:
    d=eval(df['production_countries'][index])
    b=[]
    for i in range(len(d)):
      b.append(list(d[i].values())[0])
  except:
    b=[]
  x.append(b)
df['production_countries']=x
x=[]
for index,row in df.iterrows():
  try:
    d=eval(df['cast'][index])
    b=[]
    for i in range(len(d)):
      b.append(list(d[i].values())[0])
  except:
    b=[]
  x.append(b)
df['cast']=x
total_crew=[]
for index,row in df.iterrows():
  hi=df['crew'][index]
  crew=[]
  try:
    ho=hi.rsplit("'Director', 'name': '")[1]
    he=ho.split("', ")
    crew.append(he[0])
  except:
    crew.append("N/A")

  try:
    ho=hi.rsplit("Producer\', \'name\': \'")[1]
    he=ho.split("', ")
    crew.append(he[0])
  except:
    crew.append("N/A")
  try:
    ho=hi.rsplit("'Director of Photography', 'name': '")[1]
    he=ho.split("', ")
    crew.append(he[0])
  except:
    crew.append("N/A")
  total_crew.append(crew)
total_crew
df['crew']=total_crew
df["director"] = df["crew"].str[0]
df["producer"] = df["crew"].str[1]
df["Director_of_photography"] = df["crew"].str[2]
df=df.drop('crew',1)
cols=df.columns
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()
df = df.join(pd.DataFrame(mlb.fit_transform(df.pop('genres')),
                          columns=mlb.classes_,
                          index=df.index))
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()
df = df.join(pd.DataFrame(mlb.fit_transform(df.pop('production_companies')),
                          columns=mlb.classes_,
                          index=df.index))
                          
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()
df = df.join(pd.DataFrame(mlb.fit_transform(df.pop('cast')),
                          columns=mlb.classes_,
                          index=df.index))


In [0]:
df.isnull().sum(axis=0)
df['status']=df['status'].fillna('Released')
df['runtime']=df['runtime'].fillna('93')
df['release_date']=df['release_date'].fillna('2/2/05')
df.status.unique()

array(['Released', 'Rumored', 'Post Production'], dtype=object)

In [0]:
column=list(cols)+list(studios)+list(casts)
column.remove("genres")
column.remove("production_companies")
column.remove("cast")
df['North Box Productions']='0'
df['Ronald Shusett/Gary Goldman']='0'
df['Hypnopolis']='0'
df=df[column]



In [0]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
df = df.join(pd.DataFrame(mlb.fit_transform(df.pop('production_countries')),
                          columns=mlb.classes_,
                          index=df.index))


In [0]:

from sklearn.preprocessing import LabelEncoder
lbl=LabelEncoder()
df['belongs_to_collection']=lbl.fit_transform(df['belongs_to_collection'])


In [0]:
from sklearn.preprocessing import LabelEncoder
lbl=LabelEncoder()
df['status']=lbl.fit_transform(df['status'])

In [0]:
language=pd.get_dummies(df.original_language)
language.drop('hi' ,1)
df1=pd.concat([df,language],1)
df1.drop('original_language',1, inplace=True)
dire=pd.get_dummies(df.director)
dire['Ole Bornedal']='0'
dire['Ralph Fiennes']='0'
dire['Sujoy Ghosh']='0'
dire['Stevan Filipoviƒá']='0'
dire['Jehane Noujaim']='0'
dire['Michael Dowse']='0'
dire['Hal Sutherland']='0'
dire['Joe Swanberg']='0'
dire['Phil Traill']='0'
dire['Nelson Shin']='0'
dire['Jonathan King']='0'
dire['Jong-seok Yoon']='0'





dire=dire[directors]
df1=pd.concat([df1,dire],1)
df1.drop('director',1, inplace=True)

In [0]:


pros=pd.get_dummies(df.producer)
producers=list(producers)
ranlist=['Michael Weber', 'Robby London', 'Philippa Campbell', 'Sujoy Ghosh', 'Joe Bacal', 'Branislav Jeliƒá', 'Frank Oz', 'Ant Timpson', 'Elaine J. Constantine ', 'Jason Reitman', 'Gary Binkow', 'Whitney Houston', 'Carla Fry', 'Mike Richardson', 'Christian Baute', 'David Gross']

for x in ranlist:
    producers.remove(x)




pros=pros[producers]
df1=pd.concat([df1,pros],1)
df1.drop('producer',1, inplace=True)


phos=pd.get_dummies(df.Director_of_photography)
ranlist=['Gerald Feil', 'Arlene Nelson', 'Robert Draper', 'Mihajlo Savic', 'Mathieu Vadepied', 'Scott Peck', 'Masatoshi Fukui']
dop=list(dop)
for x in ranlist:
    dop.remove(x)
phos=phos[dop]
df1=pd.concat([df1,phos],1)
df1.drop('Director_of_photography',1, inplace=True)
import datetime as dt
for i in df1.index:
       if pd.notna(df1['release_date'][i]): 
            df1['release_date'][i]=dt.datetime.strptime(df1['release_date'][i],'%m/%d/%y')                        
df1.drop("N/A",1)
df1['release_date'][i]=dt.datetime.strptime(df1['release_date'][i],'%m/%d/%y')
year = lambda x: dt.datetime.strptime(x, "%m/%d/%y" ).year
df1['year'] = df1['release_date'].map(year)
month = lambda x: dt.datetime.strptime(x, "%m/%d/%y" ).month
df1['month'] = df1['release_date'].map(month)
day = lambda x: dt.datetime.strptime(x, "%m/%d/%y" ).day
df1['day'] = df1['release_date'].map(day)
day_of_week = lambda x: dt.datetime.strptime(x, "%m/%d/%y" ).weekday()
df1['week_day']=df1['release_date'].map(day_of_week)
df1['year']=df1['year'].astype(str)
df1['month']='month' + df1['month'].astype(str)
df1['day']='day' + df1['day'].astype(str)
df1['week_day']='week_day' + df1['week_day'].astype(str)
months=pd.get_dummies(df1.month)
months.drop('month2' ,1)
df1=pd.concat([df1,months],1)
df1.drop('month',1, inplace=True)

years=pd.get_dummies(df1.year)
years.drop('2015' ,1)
df1=pd.concat([df1,years],1)
df1.drop('year',1, inplace=True)

days=pd.get_dummies(df1.day)
days.drop('day1' ,1)
df1=pd.concat([df1,days],1)
df1.drop('day',1, inplace=True)

weekdays=pd.get_dummies(df1.week_day)
weekdays.drop('week_day1' ,1)
df1=pd.concat([df1,weekdays],1)
df1.drop('week_day',1, inplace=True)      
df1.drop("release_date",1,inplace=True) 
df1 = df1[df1.budget!=0]  
Xtest=df1   
Xtest['budget']=np.log1p(Xtest['budget'])  
Xtest['popularity']=np.log1p(Xtest['popularity'])
Xtest['runtime']=np.log1p(Xtest['runtime'].astype(float))     
Xtest.head()

ValueError: ignored